# Population Health Analyst — Round 3 Practice (Google Colab)

This notebook is a **guided, hands-on simulation** of a technical take-home:
1) **Load & explore** a mock claims dataset  
2) **Run SQL-style analyses** (via SQLite) for utilization & cost  
3) **Risk stratification** with Python (Pandas)  
4) **KPIs & visualizations** (matplotlib, no seaborn)  
5) **Export artifacts** for Tableau (CSV extracts)

> ✅ Tip: Run each cell in order. If you're in **Google Colab**, use the **Upload** cell to bring in the CSV.


In [ ]:
# --- SETUP ---
# (Colab usually has these preinstalled, but we import explicitly for clarity.)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from io import StringIO

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

## 1) Get the data into this notebook

**Option A (Preferred): Upload the CSV from your computer**  
1. Download the dataset from your ChatGPT conversation: `mock_claims_data.csv`  
2. In Colab, run the next cell and use the file picker.

**Option B:** If you're not in Colab, place the CSV in the same directory as this notebook and set `CSV_PATH` manually.


In [ ]:
# --- UPLOAD (Colab) ---
# If running in Google Colab, this will open a file picker. Select mock_claims_data.csv.
CSV_PATH = "mock_claims_data.csv"  # fallback default
try:
    from google.colab import files  # will fail locally but fine in Colab
    uploaded = files.upload()       # prompts user
    if uploaded:
        CSV_PATH = list(uploaded.keys())[0]
        print("Using uploaded file:", CSV_PATH)
except Exception as e:
    print("Colab upload not available. If running locally, ensure the CSV is next to this notebook.")
    print("Using default path:", CSV_PATH)

## 2) Load & sanity-check the dataset
We'll read the claims CSV and look at schema, missingness, and a few descriptive stats.


In [ ]:
df = pd.read_csv(CSV_PATH)
print("Rows, Cols:", df.shape)
df.head()

In [ ]:
# Basic info & null checks
df.info()
print("\nMissing values per column:")
print(df.isna().sum())
print("\nQuick describe (numeric):")
display(df[['age','ed_visits','inpatient_admissions','total_cost','risk_score']].describe())

## 3) SQL-style analysis with SQLite
We'll load the DataFrame into an **in-memory SQLite database** and practice typical queries.

**Practice Prompts:**
1. Top 5% highest-cost patients  
2. Avg ED visits by chronic condition  
3. Providers with above-average inpatient admissions per 100 patients  
4. Monthly trend of total ED visits


In [ ]:
# Load into SQLite
conn = sqlite3.connect(":memory:")
df.to_sql("claims", conn, index=False, if_exists="replace")

# 1) Top 5% highest-cost patients
threshold = df['total_cost'].quantile(0.95)
q1 = f"""
SELECT patient_id, age, gender, chronic_condition, ed_visits, inpatient_admissions, total_cost, provider_id, risk_score, payer, month
FROM claims
WHERE total_cost >= {threshold};
"""
top5 = pd.read_sql(q1, conn)
print("Top 5% by total_cost (threshold >= {:.2f})".format(threshold))
top5.head(10)

In [ ]:
# 2) Avg ED visits by chronic condition
q2 = """
SELECT chronic_condition, AVG(ed_visits) AS avg_ed_visits, COUNT(*) AS n
FROM claims
GROUP BY chronic_condition
ORDER BY avg_ed_visits DESC;
"""
pd.read_sql(q2, conn)

In [ ]:
# 3) Providers with above-average inpatient admissions per 100 patients
q3 = """
WITH per_provider AS (
  SELECT provider_id,
         AVG(inpatient_admissions) AS avg_adm_per_patient,
         COUNT(*) AS patients
  FROM claims
  GROUP BY provider_id
),
summary AS (
  SELECT AVG(avg_adm_per_patient) AS network_avg
  FROM per_provider
)
SELECT p.provider_id,
       p.avg_adm_per_patient * 100.0 AS admissions_per_100,
       p.patients,
       s.network_avg * 100.0 AS network_avg_per_100
FROM per_provider p, summary s
WHERE p.avg_adm_per_patient > s.network_avg
ORDER BY admissions_per_100 DESC;
"""
pd.read_sql(q3, conn)

In [ ]:
# 4) Monthly trend of total ED visits
q4 = """
SELECT month, SUM(ed_visits) AS total_ed_visits
FROM claims
GROUP BY month
ORDER BY
  CASE month
    WHEN 'Jan' THEN 1 WHEN 'Feb' THEN 2 WHEN 'Mar' THEN 3 WHEN 'Apr' THEN 4
    WHEN 'May' THEN 5 WHEN 'Jun' THEN 6 WHEN 'Jul' THEN 7 WHEN 'Aug' THEN 8
    WHEN 'Sep' THEN 9 WHEN 'Oct' THEN 10 WHEN 'Nov' THEN 11 WHEN 'Dec' THEN 12
  END;
"""
monthly_ed = pd.read_sql(q4, conn)
monthly_ed

In [ ]:
# Plot monthly ED visits (matplotlib, single-plot, default colors)
plt.figure()
plt.plot(monthly_ed['month'], monthly_ed['total_ed_visits'], marker='o')
plt.title('Monthly ED Visits (All Patients)')
plt.xlabel('Month')
plt.ylabel('Total ED Visits')
plt.grid(True)
plt.show()

## 4) Risk stratification with Pandas

**Rule (example):**
- High Risk: `risk_score >= 2.5` **OR** `total_cost >= 20000`  
- Medium Risk: `1.0 <= risk_score < 2.5`  
- Low Risk: `risk_score < 1.0`

We'll create a tier, then compute distribution, utilization, and cost contribution.


In [ ]:
def risk_tier(row):
    if (row['risk_score'] >= 2.5) or (row['total_cost'] >= 20000):
        return 'High'
    elif row['risk_score'] >= 1.0:
        return 'Medium'
    else:
        return 'Low'

df['risk_tier'] = df.apply(risk_tier, axis=1)

tier_dist = df['risk_tier'].value_counts(normalize=True).rename('share').to_frame()
tier_util = df.groupby('risk_tier')[['ed_visits','inpatient_admissions']].mean().round(2)
tier_cost = (df.groupby('risk_tier')['total_cost'].sum() / df['total_cost'].sum()).rename('cost_share').to_frame()

summary = tier_dist.join(tier_util).join(tier_cost).sort_values('share', ascending=False)
summary

In [ ]:
# Bar plot: cost share by risk tier
tier_cost_plot = (df.groupby('risk_tier')['total_cost'].sum()
                    .sort_values(ascending=False))

plt.figure()
tier_cost_plot.plot(kind='bar')
plt.title('Cost Contribution by Risk Tier')
plt.xlabel('Risk Tier')
plt.ylabel('Total Cost')
plt.grid(True)
plt.show()

## 5) Provider performance: ED visits per 100 patients

We'll compute ED visits per 100 patients for each provider and compare to network average.


In [ ]:
prov = (df.groupby('provider_id')
          .agg(patients=('patient_id','nunique'),
               ed_total=('ed_visits','sum'),
               adm_total=('inpatient_admissions','sum'))
          .reset_index())
prov['ed_per_100'] = (prov['ed_total'] / prov['patients']) * 100.0
network_avg = prov['ed_per_100'].mean()

prov_sorted = prov.sort_values('ed_per_100', ascending=False)
print("Network average ED per 100 patients:", round(network_avg,2))
prov_sorted.head(10)

In [ ]:
# Plot provider ED per 100 (top 10)
top10 = prov_sorted.head(10)
plt.figure()
plt.bar(top10['provider_id'], top10['ed_per_100'])
plt.title('Top 10 Providers by ED Visits per 100 Patients')
plt.xlabel('Provider ID')
plt.ylabel('ED Visits per 100')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

## 6) Export extracts for Tableau (optional)
Export clean tables you might bring into Tableau for the dashboard:
- `claims_enriched.csv` (with `risk_tier`)
- `provider_summary.csv`
- `monthly_ed_trend.csv`


In [ ]:
df.to_csv('claims_enriched.csv', index=False)
prov_sorted.to_csv('provider_summary.csv', index=False)
monthly_ed.to_csv('monthly_ed_trend.csv', index=False)

print("Wrote: claims_enriched.csv, provider_summary.csv, monthly_ed_trend.csv")

## 7) Executive Summary (template)

**Top Insights (examples):**
- The top 5% highest-cost patients contribute ~X% of total spend; majority fall into **High risk** tier and are predominantly **[condition]** with **[payer]**.
- **ED visits** show a pronounced spike in **[months]**, driven by **[condition/provider]** cohorts.
- **Providers P00X & P00Y** exceed network ED-per-100 by **Δ**, suggesting targeted care management opportunities.

**Recommendations:**
- Enroll **High risk** CHF/Diabetes patients into care management & telehealth outreach.
- Implement provider-level scorecards and education for ED alternatives & care pathways.
- Stand up a monthly **utilization huddle** with clinical + operations to review cohorts and actions.

**Expected Impact:**
- ED reduction: **X–Y%** over next 2 quarters; project **$Z** savings.
- Improved HEDIS measures tied to utilization and chronic care management.


---

### You’re done!
Next steps:
- Build your **Tableau** dashboard using the exported CSVs.
- Practice presenting your **Executive Summary** aloud (≤ 5 minutes).
- Prepare answers to **stakeholder questions** (clinical & executive).
